<a href="https://colab.research.google.com/github/NickolasCrema/pyspark_curso/blob/main/pyspark_modelos_de_classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso "Spark: criando modelos de classificação"

## Preparando ambiente

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2fc789e1d0eb1c3e7eb6524b40b7e928115eac6445029cdfeea9a079eb4904cf
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import VectorAssembler

In [ ]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('Criando modelos de classificação')\
    .getOrCreate()

spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Carregando dados

In [ ]:
dados = spark.read.csv('/content/drive/MyDrive/data_spark_modelos_classificacao/dados_clientes.csv', sep=',', header=True, inferSchema=True)

In [ ]:
dados.limit(10).show(truncate=False)

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|id |Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|Internet   |SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio|MetodoPagamento |MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|0  |Nao  |0         |Sim    |Nao        |1              |Nao         |SemServicoTelefonico    |DSL        |Nao            |Sim         |Nao              |Nao           |Nao    |Nao            |Mensalmente |Sim         |BoletoEletronico|29.85  

In [ ]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



In [ ]:
dados.count()

10348

In [ ]:
dados.groupBy('Churn')\
    .count()\
    .show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



## Tratamento dos dados

In [ ]:
colunas_binarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [ ]:
todas_colunas = [f.when(f.col(i)=='Sim', 1).otherwise(0).alias(i) for i in colunas_binarias]
todas_colunas

[Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorreio = Sim) THEN 1 ELSE 0 END AS ContaCorreio'>]

In [ ]:
for coluna in reversed(dados.columns):
  if coluna not in colunas_binarias:
    todas_colunas.insert(0, coluna)

todas_colunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [ ]:
dados.select(todas_colunas).show(10, truncate=False)

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|id |Mais65anos|MesesDeContrato|Internet   |TipoContrato|MetodoPagamento |MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|0  |0         |1              |DSL        |Mensalmente |BoletoEletronico|29.85        |0    |1      |0          |0           |0                       |0              |1           |0                |0             |0      |0              |1     

In [ ]:
dataset = dados.select(todas_colunas)
dataset.show(10, truncate=False)

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|id |Mais65anos|MesesDeContrato|Internet   |TipoContrato|MetodoPagamento |MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|0  |0         |1              |DSL        |Mensalmente |BoletoEletronico|29.85        |0    |1      |0          |0           |0                       |0              |1           |0                |0             |0      |0              |1     

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



## Criando Dummies

In [ ]:
dados.select(['Internet', 'TipoContrato', 'MetodoPagamento']).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [ ]:
internet = dataset.groupBy('id')\
    .pivot('Internet')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

internet.show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [ ]:
tipo_contrato = dataset.groupBy('id')\
    .pivot('TipoContrato')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

tipo_contrato.show()

+----+--------+-----------+-----+
|  id|DoisAnos|Mensalmente|UmAno|
+----+--------+-----------+-----+
|7993|       0|          1|    0|
|8592|       0|          1|    0|
|4519|       0|          0|    1|
|1088|       0|          1|    0|
|1238|       0|          1|    0|
|1342|       1|          0|    0|
|4935|       0|          0|    1|
| 471|       0|          1|    0|
|5518|       0|          1|    0|
| 463|       0|          1|    0|
|3794|       0|          1|    0|
|9465|       0|          1|    0|
|7240|       0|          1|    0|
|9852|       0|          1|    0|
|1959|       0|          1|    0|
|7754|       0|          1|    0|
|5156|       0|          0|    1|
|6658|       0|          1|    0|
|6397|       0|          0|    1|
|1829|       0|          1|    0|
+----+--------+-----------+-----+
only showing top 20 rows



In [ ]:
metodo_pagamento = dataset.groupBy('id')\
    .pivot('MetodoPagamento')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

metodo_pagamento.show()

+-----+------+----------------+-------------+-------------+
|   id|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+-----+------+----------------+-------------+-------------+
| 3997|     0|               0|            1|            0|
| 7554|     0|               1|            0|            0|
| 6336|     0|               1|            0|            0|
| 6357|     0|               1|            0|            0|
| 9427|     0|               0|            1|            0|
| 2659|     0|               0|            1|            0|
|  471|     0|               1|            0|            0|
| 4935|     0|               0|            1|            0|
| 4818|     0|               0|            1|            0|
| 1342|     1|               0|            0|            0|
| 1959|     0|               1|            0|            0|
| 9376|     0|               0|            1|            0|
| 2366|     0|               1|            0|            0|
| 1580|     0|               0|         

In [ ]:
dataset

DataFrame[id: int, Mais65anos: int, MesesDeContrato: int, Internet: string, TipoContrato: string, MetodoPagamento: string, MesesCobrados: double, Churn: int, Conjuge: int, Dependentes: int, TelefoneFixo: int, MaisDeUmaLinhaTelefonica: int, SegurancaOnline: int, BackupOnline: int, SeguroDispositivo: int, SuporteTecnico: int, TVaCabo: int, StreamingFilmes: int, ContaCorreio: int]

In [ ]:
dataset\
    .join(internet, 'id', how='inner')\
    .join(tipo_contrato, 'id', how='inner')\
    .join(metodo_pagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('Boleto').alias('MetodoPagamento_Boleto')
    )\
    .drop(
        'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 'FibraOptica',
        'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 'DebitoEmConta',
        'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )

dataset.limit(10).show(truncate=False)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|id  |Mais65anos|MesesDeContrato|MesesCobrados    |Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
dataset = dataset\
    .join(internet, 'id', how='inner')\
    .join(tipo_contrato, 'id', how='inner')\
    .join(metodo_pagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('Boleto').alias('MetodoPagamento_Boleto')
    )\
    .drop(
        'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 'FibraOptica',
        'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 'DebitoEmConta',
        'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )

dataset.limit(10).show(truncate=False)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|id  |Mais65anos|MesesDeContrato|MesesCobrados    |Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

## Preparação dos dados

In [ ]:
dataset.limit(10).show(truncate=False)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|id  |Mais65anos|MesesDeContrato|MesesCobrados    |Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
dataset = dataset.withColumnRenamed('Churn', 'label')

In [ ]:
X = dataset.columns
X.remove('label')
X.remove('id')
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [ ]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [ ]:
dataset_prep = assembler.transform(dataset).select('features', 'label')

In [ ]:
dataset_prep.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,17,22],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

In [ ]:
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed=101)

In [ ]:
treino.count()

7206

In [ ]:
teste.count()

3142

In [ ]:
treino.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 3611|
|    0| 3595|
+-----+-----+



## Ajuste e Previsão - Regressão Logistica


In [ ]:
lr = LogisticRegression()

In [ ]:
modelo_lr = lr.fit(treino)

In [ ]:
previsoes_lr_teste = modelo_lr.transform(teste)

In [ ]:
previsoes_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1680594619286...|[0.45808374494006...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.4170949608173...|[0.19511740608882...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.14194260698794...|[0.53542619200881...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67046644011599...|[0.66160759507905...|       0.0|
|(24,[0,1,2,3,4,

### Métricas

In [ ]:
resumo_lr = modelo_lr.summary

In [ ]:
print("Acurácia: %f" % resumo_lr.accuracy)
print("Precisão: %f" % resumo_lr.precisionByLabel[1])
print("Recall: %f" % resumo_lr.recallByLabel[1])
print("F1: %f" % resumo_lr.fMeasureByLabel()[1])

Acurácia: 0.784901
Precisão: 0.770686
Recall: 0.812517
F1: 0.791049


In [ ]:
tp = previsoes_lr_teste\
    .select('label', 'prediction')\
    .where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
tn = previsoes_lr_teste\
    .select('label', 'prediction')\
    .where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
fp = previsoes_lr_teste\
    .select('label', 'prediction')\
    .where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
fn = previsoes_lr_teste\
    .select('label', 'prediction')\
    .where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

print(f'tp = {tp}\ntn = {tn}\nfp = {fp}\nfn = {fn}')

tp = 1256
tn = 1179
fp = 400
fn = 307


In [ ]:
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):
  tp = df_transform_modelo\
    .select('label', 'prediction')\
    .where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo\
      .select('label', 'prediction')\
      .where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo\
      .select('label', 'prediction')\
      .where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo\
      .select('label', 'prediction')\
      .where((f.col('label') == 1) & (f.col('prediction') == 0)).count()
  valorP = 1
  valorN = 1

  if normalize:
    valorP = tp + fn
    valorN = fp + tn

  if percentage and normalize:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*15, 'Churn', ' '*5 ,'Não-Churn')
  print(' '*4, 'Churn', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real')
  print(' '*4, 'Não-Churn', ' '*2, int(fp/valorN), ' '*7, int(tn/valorN))

In [ ]:
calcula_mostra_matriz_confusao(previsoes_lr_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179


## Ajuste e Previsão - Árvore de Decisão

In [ ]:
dtc = DecisionTreeClassifier(seed=101)

In [ ]:
modelo_dtc = dtc.fit(treino)

In [ ]:
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [ ]:
previsoes_dtc_teste.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [62.0,128.0]|[0.32631578947368...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [51.0,141.0]| [0.265625,0.734375]|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [63.0,118.0]|[0.34806629834254...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,

### Métricas

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.771483
Precisão: 0.778510
Recall: 0.755598
F1: 0.766883


In [ ]:
calcula_mostra_matriz_confusao(previsoes_dtc_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243


## Ajuste e Previsão - RandomForest

In [ ]:
rfc = RandomForestClassifier(seed=101)

In [ ]:
modelo_rfc = rfc.fit(treino)

In [ ]:
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [ ]:
previsoes_rfc_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[16.7433871675615...|[0.83716935837807...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.27313214599648...|[0.36365660729982...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[7.46885072161585...|[0.37344253608079...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[9.33276328267787...|[0.46663816413389...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.79829004739264...|[0.38991450236963...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.13263407834549...|[0.35663170391727...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[4.45872635511159...|[0.22293631775557...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.84691519125130...|[0.39234575956256...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.94796150783366...|[0.49739807539168...|       1.0|
|(24,[0,1,2,3,4,

### Métricas

In [ ]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.770210
Precisão: 0.751345
Recall: 0.804223
F1: 0.776885


In [ ]:
calcula_mostra_matriz_confusao(previsoes_rfc_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163


## Comparando os resultados dos modelos

In [ ]:
print(f"""Acurácia:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: 'accuracy'})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'accuracy'})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: 'accuracy'})}
=====================================================================================

      """
)
print(f"""Precisão:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}
=====================================================================================

      """
)
print(f"""Recall:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}
=====================================================================================

      """
)
print(f"""F1:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}
=====================================================================================

      """
)


Acurácia:
      Logistic Regression = 0.7749840865690643
      Decision Tree Classifier = 0.7714831317632082
      Random Forest Classifier = 0.7702100572883513

      
Precisão:
      Logistic Regression = 0.7584541062801933
      Decision Tree Classifier = 0.7785102175346078
      Random Forest Classifier = 0.7513448894202033

      
Recall:
      Logistic Regression = 0.8035828534868842
      Decision Tree Classifier = 0.7555982085732565
      Random Forest Classifier = 0.8042226487523992

      
F1:
      Logistic Regression = 0.7803665734700218
      Decision Tree Classifier = 0.7668831168831168
      Random Forest Classifier = 0.7768850432632881

      


In [ ]:
print('Logistic Regression:')
calcula_mostra_matriz_confusao(previsoes_lr_teste)
print('\n', '='*50)
print('\nDecision Tree:')
calcula_mostra_matriz_confusao(previsoes_dtc_teste)
print('\n', '='*50)
print('\nRandom Forest:')
calcula_mostra_matriz_confusao(previsoes_rfc_teste)

Logistic Regression:
                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179


Decision Tree:
                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243


Random Forest:
                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163


## Regressão Logistica com Cross Validation

In [ ]:
lr = LogisticRegression()

In [ ]:
grid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.01, 0.1, 0.5, 1.0, 2.0])\
    .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0])\
    .addGrid(lr.maxIter, [1, 5, 10, 20, 50])\
    .build()

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
lr_cv = CrossValidator(
    estimator = lr,
    estimatorParamMaps = grid,
    evaluator = evaluator,
    numFolds = 3,
    seed = 101
)

In [ ]:
modelo_lr_cv = lr_cv.fit(treino)

In [ ]:
previsoes_lr_cv_teste = modelo_lr_cv.transform(teste)

### Métricas

In [ ]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.776894
Precisão: 0.756548
Recall: 0.813180
F1: 0.783842


In [ ]:
calcula_mostra_matriz_confusao(previsoes_lr_cv_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1271         292
Real
     Não-Churn    409         1170


## Árvore de Decisão com Cross Validation

In [ ]:
dtc = DecisionTreeClassifier(seed=101)

In [ ]:
grid = ParamGridBuilder()\
    .addGrid(dtc.maxDepth, [2, 5, 10])\
    .addGrid(dtc.maxBins, [10, 32, 45])\
    .build()

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
dtc_cv = CrossValidator(
    estimator = dtc,
    estimatorParamMaps = grid,
    evaluator = evaluator,
    seed = 101,
    numFolds = 3
)

In [ ]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [ ]:
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

###Métricas

In [ ]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.790261
Precisão: 0.758286
Recall: 0.849008
F1: 0.801087


In [ ]:
calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1327         236
Real
     Não-Churn    423         1156


## RandomForest com Cross Validation

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
grid = ParamGridBuilder()\
    .addGrid(rfc.maxDepth, [2, 5, 10])\
    .addGrid(rfc.maxBins, [10, 32, 45])\
    .addGrid(rfc.numTrees, [10, 20, 50])\
    .build()

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
rfc_cv = CrossValidator(
    estimator = rfc,
    estimatorParamMaps = grid,
    evaluator = evaluator,
    numFolds = 3,
    seed = 101
)

In [ ]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [ ]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

### Métricas

In [ ]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.818905
Precisão: 0.795131
Recall: 0.856686
F1: 0.824761


In [ ]:
calcula_mostra_matriz_confusao(previsoes_rfc_cv_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1339         224
Real
     Não-Churn    345         1234


## Comparando modelos com Cross Validation

In [ ]:
print(f"""Acurácia:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: 'accuracy'})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_cv_dtc_teste, {evaluator.metricName: 'accuracy'})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: 'accuracy'})}
=====================================================================================

      """
)
print(f"""Precisão:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}
=====================================================================================

      """
)
print(f"""Recall:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}
=====================================================================================

      """
)
print(f"""F1:
      Logistic Regression = {evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}
      Decision Tree Classifier = {evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}
      Random Forest Classifier = {evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}
=====================================================================================

      """
)


Acurácia:
      Logistic Regression = 0.7768936982813495
      Decision Tree Classifier = 0.7902609802673456
      Random Forest Classifier = 0.8189051559516232

      
Precisão:
      Logistic Regression = 0.756547619047619
      Decision Tree Classifier = 0.7582857142857143
      Random Forest Classifier = 0.7951306413301663

      
Recall:
      Logistic Regression = 0.8131797824696098
      Decision Tree Classifier = 0.8490083173384517
      Random Forest Classifier = 0.8566858605246321

      
F1:
      Logistic Regression = 0.7838421214924453
      Decision Tree Classifier = 0.801086628433444
      Random Forest Classifier = 0.8247613181398215

      


In [ ]:
print('Logistic Regression:')
calcula_mostra_matriz_confusao(previsoes_lr_cv_teste)
print('\n', '='*50)
print('\nDecision Tree:')
calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste)
print('\n', '='*50)
print('\nRandom Forest:')
calcula_mostra_matriz_confusao(previsoes_rfc_cv_teste)

Logistic Regression:
                     Previsto
                Churn       Não-Churn
     Churn        1271         292
Real
     Não-Churn    409         1170


Decision Tree:
                     Previsto
                Churn       Não-Churn
     Churn        1327         236
Real
     Não-Churn    423         1156


Random Forest:
                     Previsto
                Churn       Não-Churn
     Churn        1339         224
Real
     Não-Churn    345         1234


## Modelo final

In [ ]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [ ]:
print(f'Best Max Depth: {melhor_modelo_rfc_cv.getMaxDepth()}')
print(f'Best Max Bins: {melhor_modelo_rfc_cv.getMaxBins()}')
print(f'Best Num Trees: {melhor_modelo_rfc_cv.getNumTrees}')

Best Max Depth: 10
Best Max Bins: 45
Best Num Trees: 50


In [ ]:
rfc_tuning = RandomForestClassifier(maxDepth=10, maxBins=45, numTrees=10, seed=101)

In [ ]:
modelo_rfc_tuning = rfc_tuning.fit(dataset_prep)

In [ ]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [ ]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

In [ ]:
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [ ]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [ ]:
novo_cliente_prep = assembler.transform(novo_cliente).select('features')

In [ ]:
novo_cliente_prep.show(truncate=False)

+----------------------------------------------------------------------------+
|features                                                                    |
+----------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------+



In [ ]:
previsao = modelo_rfc_tuning.transform(novo_cliente_prep)

In [ ]:
previsao.select('prediction').show()

+----------+
|prediction|
+----------+
|       1.0|
+----------+

